In [ ]:
from evaluate_thresholds import (
    read_json,
    get_scores,
    plot_accuracy_vs_threshold,
    check_labels,
)

In [ ]:
INPUT_FILE = "debug-labeled-pdps.json"

In [ ]:
curves = read_json(INPUT_FILE)
df = get_scores(curves)

In [ ]:
plot_accuracy_vs_threshold(df)

In [ ]:
best = df.iloc[df["accuracy"].idxmax()]

In [ ]:
best

In [ ]:
check_labels(curves, best["labels"])